In [2]:
# Importando bibliotecas

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import io
import csv
from datetime import *


StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 4, Finished, Available, Finished)

In [3]:
# Formatação do tipo Data 

def log():
  return datetime.now().strftime('%Y-%m-%d %H:%M:%S') + ' >>>'


# Incluir variável processamento

dt_proc = datetime.now().strftime('%Y%m%d%H%M%S')
dt_proc

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 5, Finished, Available, Finished)

'20250129210549'

In [4]:
# Leitura  dos arquivos CSV na pasta Pagamentos da camada Raw

df = spark.read.option('header', 'true').csv("Files/Raw/Pagamentos/*.csv")

df.createOrReplaceTempView("df")

display(df)

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c16831c8-bde7-403b-8ac1-1a95a1679e00)

In [5]:
# Quantidade de Registros

qtd_registros = df.count()
print(log(),f'Quantidade de Registros: {qtd_registros}')

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 7, Finished, Available, Finished)

2025-01-29 21:05:54 >>> Quantidade de Registros: 90774


In [6]:
df.printSchema()

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 8, Finished, Available, Finished)

root
 |-- id_pagamento: string (nullable = true)
 |-- id_fatura: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- data_pagamento: string (nullable = true)
 |-- valor_pagamento: string (nullable = true)



In [7]:
df_pagamento_formated = spark.sql(f"""
            SELECT
              CAST(
                    CONCAT( REPLACE( SUBSTRING( id_pagamento, 4, LENGTH(id_pagamento)), '-', '')) as BIGINT) AS id_pagamento,
              CAST( REGEXP_REPLACE(id_fatura, '-', '') as BIGINT) as id_fatura,
              CAST(id_cliente as BIGINT) as id_cliente,
              "{dt_proc}" as dt_proc,
              SUBSTRING( REPLACE( data_pagamento, '-', ''), 1, 6) as Ref,
              TO_DATE( data_pagamento ) as data_pagamento,
              CAST( valor_pagamento as DECIMAL(10,2)) as valor_pagamento
            FROM df

""")
df_pagamento_formated.createOrReplaceTempView('df_pagamento_formated')

display(df_pagamento_formated)

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6efa7258-a839-46bf-9c09-940082b6826e)

In [8]:
spark.sql("""
      SELECT
            Ref
      FROM df_pagamento_formated
      GROUP BY 1
      ORDER BY 1
""").show()

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 10, Finished, Available, Finished)

+------+
|   Ref|
+------+
|202301|
|202302|
|202303|
|202304|
|202305|
|202306|
|202307|
|202308|
|202309|
|202310|
|202311|
|202312|
|202401|
|202402|
+------+



In [9]:
# Salvar arquivo na pasta Trusted

df_pagamento_formated.write.mode('append').partitionBy('Ref').parquet("Files/Trusted//tb_pagamento")

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 11, Finished, Available, Finished)

**Início Carregamento Base Controle**

In [10]:
df_controle_pagamento = spark.sql(f"""
            SELECT
                'tb_pagamento' as tabela_name,
                "{dt_proc}" as dt_proc,
                {qtd_registros} as qtd_registros
""")
df_controle_pagamento.write.mode('append').parquet("Files/Controle/tb_pagamento")

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 12, Finished, Available, Finished)

In [11]:
# Salvar  arquivo de controle de processamento

df_controle_pagamento.write.mode('append').parquet("Files/Controle/tb_pagamento")

StatementMeta(, cfad4691-e90c-42ba-87e3-89310f4e64c3, 13, Finished, Available, Finished)